<a href="https://colab.research.google.com/github/WhiteAndBlackFox/nlp/blob/EM/%D0%A2%D0%B5%D0%BC%D0%B0%D1%82%D0%B8%D1%87%D0%B5%D1%81%D0%BA%D0%BE%D0%B5_%D0%BC%D0%BE%D0%B4%D0%B5%D0%BB%D0%B8%D1%80%D0%BE%D0%B2%D0%B0%D0%BD%D0%B8%D0%B5_EM_%D0%B0%D0%BB%D0%B3%D0%BE%D1%80%D0%B8%D1%82%D0%BC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Тематическое моделирование. EM-алгоритм

## Импортируем библиотеки

In [ ]:
!pip install annoy
!pip install pymorphy2
!pip install pyLDAvis

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 646 kB 12.8 MB/s 
  Created wheel for annoy: filename=annoy-1.17.0-cp37-cp37m-linux_x86_64.whl size=391586 sha256=89abf30d5adf172ec2ed1c7908c09e9a36b65a085c118c92d95a39071e84bba8
  Stored in directory: /root/.cache/pip/wheels/4f/e8/1e/7cc9ebbfa87a3b9f8ba79408d4d31831d67eea918b679a4c07
Successfully built annoy
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 55 kB 3.7 MB/s 
     |████████████████████████████████| 8.2 MB 28.7 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.7 MB 9.2 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
  Created wh

In [ ]:
import re
import pandas as pd
import numpy as np
import annoy

import nltk
from nltk.corpus import stopwords
from pymorphy2 import MorphAnalyzer
from pymystem3 import Mystem
from string import punctuation

from gensim.models import *
from gensim import corpora
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
import matplotlib.pyplot as plt
import warnings

nltk.download('stopwords')
warnings.simplefilter("ignore")
warnings.warn("deprecated", DeprecationWarning)

/usr/local/lib/python3.7/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Iterable
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


## 1. Объединить в одну выборку


In [ ]:
#@title Скачиваем данные
!wget https://www.dropbox.com/s/fnpq3z4bcnoktiv/positive.csv
!wget https://www.dropbox.com/s/r6u59ljhhjdg6j0/negative.csv

--2022-06-15 16:46:31--  https://www.dropbox.com/s/fnpq3z4bcnoktiv/positive.csv
Resolving www.dropbox.com (www.dropbox.com)... 162.125.65.18, 2620:100:6021:18::a27d:4112
Connecting to www.dropbox.com (www.dropbox.com)|162.125.65.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/fnpq3z4bcnoktiv/positive.csv [following]
--2022-06-15 16:46:31--  https://www.dropbox.com/s/raw/fnpq3z4bcnoktiv/positive.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc6ae2711b3dc64eca3e5c06b15b.dl.dropboxusercontent.com/cd/0/inline/BnTElrhLQUVugBIVEfhJvdPT_7hxn5SNy6SSltmYqZipGEEAZyzNG2iFWtj5Bn0GyY4ZDkZunUQ307p1D4Gs7qouT8lf8Ul7bAKR1m9pfVxrlkJRuwnq0Iwg_uTnrSE-WY1m-y4WMS-eNhfJzEoywp6eYDfVGPLwmIsdHCL4iiQ0hg/file# [following]
--2022-06-15 16:46:32--  https://uc6ae2711b3dc64eca3e5c06b15b.dl.dropboxusercontent.com/cd/0/inline/BnTElrhLQUVugBIVEfhJvdPT_7hxn5SNy6SSltmYqZipGEEAZyzNG2iFWtj5Bn0Gy

In [ ]:
pos = pd.read_csv('positive.csv', sep=";",  usecols=[3], names=['tweet'])
pos['label'] = ['positive'] * len(pos)
neg = pd.read_csv('negative.csv', sep=";",  usecols=[3], names=['tweet'])
neg['label'] = ['negative'] * len(neg)
df = pos.append(neg)
df = df.reset_index().drop('index', axis=1)
watch_df = df.iloc[np.r_[0:5, -5:0]]
print(watch_df)

                                                    tweet     label
0       @first_timee хоть я и школота, но поверь, у на...  positive
1       Да, все-таки он немного похож на него. Но мой ...  positive
2       RT @KatiaCheh: Ну ты идиотка) я испугалась за ...  positive
3       RT @digger2912: "Кто то в углу сидит и погибае...  positive
4       @irina_dyshkant Вот что значит страшилка :D\nН...  positive
226829  Но не каждый хочет что то исправлять:( http://...  negative
226830  скучаю так :-( только @taaannyaaa вправляет мо...  negative
226831          Вот и в школу, в говно это идти уже надо(  negative
226832  RT @_Them__: @LisaBeroud Тауриэль, не грусти :...  negative
226833  Такси везет меня на работу. Раздумываю приплат...  negative


## 2. Проведем исследование и выявим тематики о которых говорят в твитах


In [ ]:
emoji_list = [":)",":‑)",":-]",":-3",":->","8-)",":-}",":o)",":c)",":^)","=]","=)","<3",":-(",":(",":c",":<",":[",">:[",":{",">:(",":-c",":-< ",":-[",":-||", "о_о"]
morph = MorphAnalyzer()

# Стоп слова нужны на русском
stop_words = set(stopwords.words("russian"))

In [ ]:
def del_emoji(words):
  '''
    Функция для удаления текстовых эмоций
  '''
  for el in emoji_list:
    words = words.replace(el, '')
  return words

def preprocessing_text(txt: str):
  '''
    Предобработка текста, т.е. удаляем лишнее, заменяем и т.д.
  '''
  # Изначально все в нижний регистр переводим
  txt = txt.lower()
  # Удаляем всех пользователей: @user
  txt = re.sub(r"@[\w]*", '', txt)
  # Удаляем эмодзи, чтобы не терялась смысловая нагрузка надо бы их конечно перобразовать в слова, на подобии: :) -> "счастлив" или :-( -> "печально", но мы это опустим
  txt = del_emoji(txt)
  # Удаляем всю пунктуацию
  txt = re.sub(r'[^\w\s]', ' ', txt)
  # Заменим спец. символы c цифрами на пробелы
  txt = re.sub(r'[^a-zA-Zа-яёA-ЯЁ]', ' ', txt)
  # Удалим из текста слова длиной менее 2-х символов
  txt = [w for w in txt.split() if len(w) > 2]
  # Удалим стоп-слова
  txt = [w for w in txt if w not in stop_words] 
  # Приведем к нормальной форме
  return [morph.parse(w)[0].normal_form for w in txt]  
  

In [ ]:
#@title Проводим чистку данных
df['preprocessing'] = df.tweet.apply(preprocessing_text)
df.reset_index(drop=True)
df.head()

,tweet,label,preprocessing
0,"@first_timee хоть я и школота, но поверь, у на...",positive,"[школотый, поверь, самый, общество, профилиров..."
1,"Да, все-таки он немного похож на него. Но мой ...",positive,"[таки, немного, похожий, мальчик, равно]"
2,RT @KatiaCheh: Ну ты идиотка) я испугалась за ...,positive,"[идиотка, испугаться]"
3,"RT @digger2912: ""Кто то в углу сидит и погибае...",positive,"[угол, сидеть, погибать, голод, ещё, порция, в..."
4,@irina_dyshkant Вот что значит страшилка :D\nН...,positive,"[значит, страшилка, блин, посмотреть, часть, с..."


In [ ]:
# Формируем словарь уникальных токенов
dictionary = corpora.Dictionary(df['preprocessing'])
dictionary.filter_extremes(no_below = 10, no_above = 0.9, keep_n=None)
dictionary.save('tweet.dict')

In [ ]:
# Векторизация твитов
corpus = [dictionary.doc2bow(text) for text in df['preprocessing']]
corpora.MmCorpus.serialize('tweet.model', corpus)

In [ ]:
# Обучаем модель
%%time
lda = ldamodel.LdaModel(corpus, id2word=dictionary, num_topics=10)

CPU times: user 38.7 s, sys: 643 ms, total: 39.4 s
Wall time: 38.8 s


In [ ]:
# Смотрим что получилось
lda.show_topics(num_topics=10, num_words=5, formatted=True)

[(0,
  '0.046*"спать" + 0.024*"хотеть" + 0.020*"http" + 0.016*"сон" + 0.014*"учить"'),
 (1,
  '0.044*"завтра" + 0.029*"школа" + 0.028*"сегодня" + 0.026*"хотеть" + 0.020*"идти"'),
 (2,
  '0.035*"это" + 0.019*"http" + 0.017*"видеть" + 0.015*"очень" + 0.014*"такой"'),
 (3,
  '0.070*"день" + 0.018*"самый" + 0.017*"хороший" + 0.017*"это" + 0.017*"весь"'),
 (4,
  '0.049*"http" + 0.045*"любить" + 0.016*"такой" + 0.016*"хотеть" + 0.015*"мой"'),
 (5,
  '0.032*"болеть" + 0.025*"скучать" + 0.019*"блин" + 0.019*"http" + 0.019*"это"'),
 (6,
  '0.027*"хотеть" + 0.021*"знать" + 0.021*"это" + 0.018*"бля" + 0.015*"всё"'),
 (7,
  '0.016*"утро" + 0.016*"http" + 0.014*"дело" + 0.013*"найти" + 0.013*"мочь"'),
 (8,
  '0.023*"это" + 0.020*"amp" + 0.016*"очень" + 0.015*"http" + 0.014*"блядь"'),
 (9,
  '0.048*"год" + 0.031*"новый" + 0.029*"кто" + 0.025*"жаль" + 0.016*"http"')]

## 3. Сделаем визуализацию кластеров тематик


In [ ]:
vis_data = gensimvis.prepare(lda, corpus, dictionary)
pyLDAvis.display(vis_data)

## 4. Проинтерпритируем получившиеся тематики

В результате визуализации у нас получилось, что:

**Тема 1.** Учеба

**Тема 2.** Погода

**Тема 3.** Желание

**Тема 4.** Чувства и эмоции

**Тема 5.** Новый год

**Тема 6.** Болезнь

**Тема 7.** Утро

**Тема 8.** Попалось amp - это может быть как эмоция, так и сокразение названии компании

**Тема 9.** Любовь

**Тема 4.** Сон